https://medium.com/analytics-vidhya/ohmygraphs-graphsage-in-pyg-598b5ec77e7b

In [1]:
import torch
import torch_geometric.transforms as T
from torch_geometric.datasets import Reddit
from ogb.nodeproppred import PygNodePropPredDataset
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.loader import NeighborLoader

In [2]:
# Use GPU if available
device = f'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
print(device)

cpu


In [3]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


# Make model

Initialization:
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.SAGEConv.html

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dimension, out_dimension, dropout, aggr='mean', normalization = True, activation_function = True, bias = True):
      # TODO: maybe activation_function should be False
      # TODO: what should hidden_dimension be?

        super().__init__()
        # as K = 2, we have 2 layers
        self.dropout = dropout
        self.conv1 = SAGEConv(in_channels = in_dimension, out_channels = in_dimension, project = activation_function, bias = bias)
        self.conv2 = SAGEConv(in_channels = in_dimension, out_channels = out_dimension, normalize = normalization, project = activation_function, bias = bias)

    #def forward(self, matrix_nodes_feature, matrix_adj):
    def forward(self, data):
      # matrix_nodes_features = rows == nodes; columns == features
      # matrix_adj = nodes * nodes
      # data.x gives matrix where row = nodes, columns = feature
        data = data.cuda()
        #x = self.conv1(matrix_nodes_feature, matrix_adj)
        x = self.conv1(data.x, data.adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout)

        x = self.conv2(x, data.adj_t)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout)

        return torch.log_softmax(x, dim=-1)

# Data

In [ ]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv',
                                 transform=T.ToSparseTensor())
data = dataset[0]

Split data

In [ ]:
# this dataset comes with train-val-test splits predefined for benchmarking
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [ ]:
print(f' dataset has {data.num_nodes} nodes where each node has a {data.num_node_features} dim feature vector')
print(f' dataset has {data.num_edges} edges where each edge has a {data.num_edge_features} dim feature vector')
print(f' dataset has {dataset.num_classes} classes')

In [ ]:
(data.adj_t).shape

Check data into trin, validation and test

In [ ]:
print(split_idx['train'].shape)
print(split_idx['valid'].shape)
print(split_idx['test'].shape)

# Data 2.0 Reddit

In [ ]:
dataset = Reddit("./")
data = dataset[0]

In [ ]:
train_idx = data.train_mask
val_idx = data.val_mask
test_idx = data.test_mask
# Accessing feature and label data for each split
X_train, y_train = data.x[train_idx], data.y[train_idx]
X_val, y_val = data.x[val_idx], data.y[val_idx]
X_test, y_test = data.x[test_idx], data.y[test_idx]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
data = data.to(device, 'x', 'y')
data

# Train method

In [ ]:
def train(model, data, train_idx, optimizer):
  #TODO: check when to use validation set
    model.train()

    optimizer.zero_grad()
    out = model(data)[train_idx]
    print (out.shape)
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx]) #TODO: check for correct loss function
    loss.backward()
    optimizer.step()

    return loss.item()

# Test method
TODO: check this shit

In [ ]:
@torch.no_grad() # make sure that model doesn't change gradient
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data) # forward pass
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

# Hyperparameters

In [ ]:
learning_rate = 0.0001
epochs = 10
output_dim = data.num_node_features
aggregator = 'mean' # variable to change/play around with for experiments
dropout_rate = 0.4
normalization = True
activation_function = True
bias = True
batch =  512
neighborhood_1 = 25
neighborhood_2 = 10

In [ ]:
evaluator = Evaluator(name='ogbn-arxiv')

# Create model

In [ ]:
model = GraphSAGE(in_dimension = data.num_node_features,
                 out_dimension = output_dim,
                  dropout= dropout_rate,
                  aggr = aggregator,
                  normalization = normalization,
                  activation_function = activation_function,
                  bias = bias)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create loader

In [ ]:
#from torch_sparse import SparseTensor

#edge_index = data.edge_index
#adj = SparseTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(data.num_nodes, data.num_nodes))

# Update your data object
#data.adj_t = adj

#kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}
#train_loader = NeighborLoader(data, input_nodes=data.train_mask,
#                              num_neighbors=[25, 10], shuffle=True, **kwargs)

loader = NeighborLoader(
    data,
    input_nodes = data.train_mask,
    # Sample neighbors for each node for 2 iterations
    num_neighbors =[neighborhood_1, neighborhood_2],
    batch_size = batch,
    shuffle = True
)

In [ ]:
kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}
train_loader = NeighborLoader(data, input_nodes=data.train_mask,
                              num_neighbors=[25, 10], shuffle=True, **kwargs)

# Train model

In [ ]:
for epoch in range(1, 1 + epochs):
    for batch in train_loader:
        optimizer.zero_grad
        train(model,batch,batch_idx,optimizer)

        #optimizer.zero_grad()
        #y = batch.y[:batch.batch_size]
        #y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        #loss = F.cross_entropy(y_hat, y)
        #loss.backward()
        #optimizer.step()

    #loss = train(model, data, train_idx, optimizer)
    #result = test(model, data, split_idx, evaluator)
    #logger.add_result(run, result)

        #train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch}/{epochs}, '
          f'Loss: {loss:.4f}, ')
              #f'Train: {100 * train_acc:.2f}%, '
              #f'Valid: {100 * valid_acc:.2f}% '
              #f'Test: {100 * test_acc:.2f}%')

In [ ]:
total = 0
correct = 0
with torch.no_grad():   # No need for keepnig track of necessary changes to the gradient.
  for data in test_dl:
    X, y = data
    output = net(X)
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

In [ ]:
#GPT

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import NeighborSampler
from torch_geometric.datasets import Planetoid

# Load a dataset (e.g., Cora)
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


# Create the model with the number of features and classes from the dataset
model = GraphSAGE(dataset.num_node_features, 128, dataset.num_classes)

# Define the NeighborSampler
loader = NeighborSampler(data.edge_index, node_idx=None,
                         sizes=[10, 10], batch_size=128,
                         shuffle=True, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)

# Move model to GPU if available, set to training mode
model.train()

# Training loop
for batch_size, n_id, adjs in loader:
    # `adjs` is a list of `(edge_index, e_id, size)` tuples.
    adjs = [adj.to(device) for adj in adjs]  # Move adjs to the correct device

    optimizer.zero_grad()
    out = model(data.x[n_id], adjs)  # Perform a single forward pass.
    loss = F.nll_loss(out, data.y[n_id[:batch_size]])  # Compute the loss solely based on the nodes in the current batch.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.

    print(f'Loss: {loss.item()}')


In [ ]:
(data.x).shape
 # node features matrix, where: row = node, column = feature.


In [ ]:
from torch_geometric.datasets import Reddit

In [ ]:
dataset = Reddit("./")

In [ ]:
data = dataset[0]

In [ ]:
print(data.x) #nodes features, where row = node's feature vector.
print()
print(data.y) #nodes labels
print()
print(data.edge_index) #adjacency information

In [ ]:
G = to_networkx(data, to_undirected=True)

In [ ]:
plt.figure(figsize=(12, 8))
nx.draw(G, with_labels=True, node_color=[[.7, .7, .7]])
plt.show()